# Deploy and execute a GraphStorm SageMaker Pipeline

Next, you can create the SageMaker pipeline to run the steps needed to train a GraphStorm model. GraphStorm provides helper scripts to create a SageMaker pipeline. To run the example, you would need to have cloned the GraphStorm repository to get access to the pipeline creation and execution scripts, which you should have done in notebook 0, with `git clone https://github.com/awslabs/graphstorm.git $HOME/graphstorm-demo`

In [ ]:
import logging

logging.basicConfig(level=logging.INFO, force=True)
logging.getLogger("boto3").setLevel(logging.WARNING)
logging.getLogger("botocore").setLevel(logging.WARNING)
logging.getLogger("aibotocore").setLevel(logging.WARNING)
logging.getLogger("fsspec").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("s3transfer").setLevel(logging.WARNING)

## Prepare and execute pipeline

As first step, extract the information for the data outputs you created in notebook 0-Data-Preparation. You will use this information to set up your SageMaker GraphStorm pipeline.

In [ ]:
import json
from pprint import pp

# Read information about the graph from the JSON file you created in notebook 0
with open("task-info.json", "r") as f:
    task_info = json.load(f)
pp(task_info)

GRAPH_NAME=task_info["GRAPH_NAME"]
BUCKET=task_info["BUCKET"]
GRAPH_ID=task_info["GRAPH_ID"]
EXPORTED_GRAPH_S3 = task_info["EXPORTED_GRAPH_S3"]
GCONSTRUCT_CONFIG = task_info["GCONSTRUCT_CONFIG"]
GS_HOME = task_info["GS_HOME"]

# The pipeline's SageMaker execution role needs the following permissions:
# - AmazonSageMakerFullAccess
# - AmazonS3FullAccess (or more restricted S3 access as needed, needs read/write to your $BUCKET)
# - AWS KMS permissions if using encrypted S3 buckets
# For detailed permissions, see: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html
SM_EXECUTION_ROLE = "arn:aws:iam::123456789012:role/service-role/<Your-SageMaker-Job-Execution-Role>"


TRAIN_YAML = "ieee_cis_nc_training.yaml"
INFERENCE_YAML = "ieee_cis_nc_inference.yaml"
YAML_S3_PREFIX = f"s3://{BUCKET}/yaml/{GRAPH_NAME}"

In [ ]:
# Set a name for the pipeline, needs to be unique per region
PIPELINE_NAME = f"{GRAPH_NAME}-graphstorm-pipeline-{GRAPH_ID}"

You will need to have the training and inference YAML files available on S3 as well.

In [ ]:
!aws s3 cp $TRAIN_YAML $YAML_S3_PREFIX/$TRAIN_YAML
!aws s3 cp $INFERENCE_YAML $YAML_S3_PREFIX/$INFERENCE_YAML

The training and inference files are mostly the same. The main difference for the inference configuration is that it enables running inference on all target nodes using `infer_all_target_nodes: true`.

This produces embeddings for all `Transaction` nodes. To avoid biased evaluation when including nodes from the training set, we also need to disable running evaluation, setting `no_validation: true` for inference.

### Deploy pipeline

To make pipeline creation easier, we provide a bash script, `deploy_recommendations_pipeline.sh`, that fills in some of the parameters needed to deploy a pipeline. 

You will need to provide your input data location (`--input-s3`) and YAML file to use for training and inference (`--yaml-s3`). 

The script will also require providing a SageMaker execution role ARN configured with S3 access. 

> **Note about instance type quotas**: The default configuration uses `ml.m5.4xlarge` instances. Your AWS account may have a default quota of 0 for this instance type. If you encounter quota issues, you have several options:
> 1. [Request a quota increase through the AWS Service Quotas console](https://docs.aws.amazon.com/sagemaker/latest/dg/regions-quotas.html#regions-quotas-quotas)
> 2. Use alternative instance types with similar specifications, for which you already have quotas for.
>
>To modify the instance type, provide the `--instance-type` argument to the `deploy_fraud_detection_pipeline.sh` script.

Once all parameters are provided, the script will deploy the SageMaker pipeline. For more details on how to deploy GraphStorm SageMaker pipelines, see the [GraphStorm documentation](https://graphstorm.readthedocs.io/en/latest/advanced/sagemaker-pipelines.html), and contents of `deploy_fraud_detection_pipeline.sh` as an example.

In [ ]:
!bash deploy_fraud_detection_pipeline.sh \
    --bucket-name $BUCKET \
    --config-filename $GCONSTRUCT_CONFIG \
    --execution-role $SM_EXECUTION_ROLE \
    --graphstorm-location $GS_HOME \
    --input-s3 $EXPORTED_GRAPH_S3 \
    --pipeline-name $PIPELINE_NAME \
    --train-yaml-s3 $YAML_S3_PREFIX/$TRAIN_YAML \
    --inference-yaml-s3 $YAML_S3_PREFIX/$INFERENCE_YAML

This will create a SageMaker pipeline named `f"{GRAPH_NAME}-graphstorm-pipeline-{GRAPH_ID}"`, which you can inspect by navigating to the **[SageMaker Studio UI](https://us-east-1.console.aws.amazon.com/sagemaker/home?region=us-east-1#/studio-landing)**, selecting your Domain and user profile that you used when deploying the pipeline, and selecting **Open Studio**. 

In the navigation pane, choose **Pipelines**. There should be a pipeline named `ieee-cis-fraud-detection-graphstorm-pipeline-<GRAPH_ID>`. Choose the pipeline, which will take you to the **Executions** tab for the pipeline. Choose **Graph** to view the pipeline steps.


<img src="images/sm-pipeline.png" width="25%">

## Execute Pipeline

The GraphStorm pipeline has the following steps:

* GConstruct that takes the input tabular graph data, processes node identifiers, features and labels, and partitions the graph to prepare
it for distributed training
* Training step that trains the classification model and produces the output model and any learnable embeddings on S3
* An inference step that produces the GNN embeddings and predictions for every target node type (`Transaction`) in the graph and uploads them to S3

To execute the pipeline you can run:


In [ ]:
# Set a specific subpath to have for the execution
import time

timestamp = f"{time.time():.0f}"
EXECUTION_SUBPATH = f"gs-pipeline-example-{timestamp}"

In [ ]:
!python3 $GS_HOME/sagemaker/pipeline/execute_sm_pipeline.py \
    --pipeline-name $PIPELINE_NAME \
    --region us-east-1 \
    --execution-subpath $EXECUTION_SUBPATH

The execution will take around 25 minutes. You can monitor the progress of the execution in SageMaker Studio Pipelines UI, in the **Executions** tab for the pipeline.

Once complete the pipeline will create embeddings for `Transaction` nodes in a pre-determined location on S3, defined in the `deploy_recommendations_pipeline.sh` script:

In [ ]:
EMB_S3 = f"s3://{BUCKET}/pipelines-output/{PIPELINE_NAME}/{EXECUTION_SUBPATH}/inference/embeddings/"

In [ ]:
!aws s3 ls $EMB_S3

You will see that the inference job has created embedding files for every Transaction node in the graph, as well as a JSON file that describes the generated embeddings.

In [ ]:
PRED_S3 = f"s3://{BUCKET}/pipelines-output/{PIPELINE_NAME}/{EXECUTION_SUBPATH}/inference/predictions/"

In [ ]:
!aws s3 ls $PRED_S3

## Enrich exported graph files with GNN embeddings

With the GNN node embeddings now available, you can enrich your original graph data with the GNN embeddings. To do so you can again use a function from the `NeptuneGS` library, providing the path to your original exported data, and the location of the embeddings on S3:


In [ ]:
from os import path as osp
from neptune_gs.attach_gs_data import attach_gs_data_to_na

# Ensure that the enriched files are created under the same prefix as the original data
ENRICHED_S3 = osp.join(EXPORTED_GRAPH_S3, "enriched")

In [ ]:
attach_gs_data_to_na(
    input_s3=EXPORTED_GRAPH_S3,
    output_s3=ENRICHED_S3,
    embeddings_s3=EMB_S3,
    predictions_s3=PRED_S3,
)

The above script will join your original graph with the generated embeddings and predictions based on node id, and create new node output files that include new columns named `embeddings:Vector` and `pred:Float[]` that you will use in the following notebooks to enrich the graph data.

By placing the enriched data under the same prefix as the original data, we can import all data using a single [Neptune Analytics import task](https://docs.aws.amazon.com/neptune-analytics/latest/userguide/loading-data-existing-graph.html).